In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import cv2
from tensorflow.keras import layers
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [2]:
img = cv2.imread("Cat1/1.jpg")
type(img)

numpy.ndarray

In [3]:
path_cat = 'PetImages/Cat/'
path_dog = 'PetImages/Dog/'

In [4]:

data = []
labels = []


In [ ]:
#usunąć zdjecia z folderów (na trello w notatkach lsita zdjeci które się nie wczytują)

In [1]:
#tutaj dodac generator

In [5]:
cat_images = os.listdir(path_cat)
for image_name in cat_images:
    try:
        image_path = os.path.join(path_cat, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (150, 150))
        image = image.astype('float32') / 255.0
        data.append(image)
        labels.append('cat')
    except:
        pass


PetImages/Cat/10125.jpg
PetImages/Cat/10404.jpg
PetImages/Cat/10501.jpg
PetImages/Cat/10820.jpg
PetImages/Cat/11210.jpg
PetImages/Cat/11565.jpg
PetImages/Cat/11874.jpg
PetImages/Cat/11935.jpg
PetImages/Cat/140.jpg
PetImages/Cat/2663.jpg
PetImages/Cat/3300.jpg
PetImages/Cat/3491.jpg
PetImages/Cat/4833.jpg
PetImages/Cat/5553.jpg
PetImages/Cat/660.jpg
PetImages/Cat/666.jpg
PetImages/Cat/7968.jpg
PetImages/Cat/7978.jpg
PetImages/Cat/8470.jpg
PetImages/Cat/850.jpg
PetImages/Cat/9171.jpg
PetImages/Cat/936.jpg
PetImages/Cat/9565.jpg
PetImages/Cat/9778.jpg
PetImages/Cat/Thumbs.db


In [7]:
dog_images = os.listdir(path_dog)
for image_name in dog_images:
    try:
        image_path = os.path.join(path_dog, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (150, 150))
        image = image.astype('float32') / 255.0
        data.append(image)
        labels.append('dog')
    except:
        pass


PetImages/Dog/10158.jpg
PetImages/Dog/10401.jpg
PetImages/Dog/10747.jpg
PetImages/Dog/10797.jpg
PetImages/Dog/11410.jpg
PetImages/Dog/11675.jpg
PetImages/Dog/11702.jpg
PetImages/Dog/11849.jpg
PetImages/Dog/11853.jpg
PetImages/Dog/1308.jpg
PetImages/Dog/1866.jpg
PetImages/Dog/2384.jpg
PetImages/Dog/2688.jpg
PetImages/Dog/2877.jpg
PetImages/Dog/3136.jpg
PetImages/Dog/3288.jpg
PetImages/Dog/3588.jpg
PetImages/Dog/4367.jpg
PetImages/Dog/5604.jpg
PetImages/Dog/5736.jpg
PetImages/Dog/6059.jpg
PetImages/Dog/6238.jpg
PetImages/Dog/6718.jpg
PetImages/Dog/7112.jpg
PetImages/Dog/7133.jpg
PetImages/Dog/7369.jpg
PetImages/Dog/7459.jpg
PetImages/Dog/7969.jpg
PetImages/Dog/8730.jpg
PetImages/Dog/9188.jpg
PetImages/Dog/Thumbs.db


In [8]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [10]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [11]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))


20100
5025
20100
5025


In [12]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

In [13]:
# Dodawanie warstw gęstych
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Warstwa wyjściowa z dwoma neuronami (kot i pies)

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[2654208,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:StatelessRandomUniformV2]

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
results = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
print(classification_report(y_test.argmax(axis=1), model.predict(X_test).argmax(axis=1)))

In [ ]:
plt.plot(results.history["loss"])
plt.title("loss")
plt.show()

In [ ]:
plt.plot(results.history["accuracy"])
plt.title("accuracy")
plt.show()